# Short Circuit Example

This notebook contains an example of a short circuit calculation using `power-grid-model`.
The following points are covered
 * Construction of the model
 * Run short circuit calculation, and its relevant function arguments

This notebook skips most of the common features across all calculations of power-grid-model like updating model, batch calculations and error handling.

It serves as an example of how to use the Python API. For detailed API documentation, refer to
[Python API reference](../api_reference/python-api-reference.md)
and [Native Data Interface](../advanced_documentation/native-data-interface.md).


## Example Network

We use a simple network with 3 nodes, 1 source, 3 lines, and 2 loads. As shown below:

```
 -----------------------line_8---------------
 |                                          |
node_1 ---line_3--- node_2 ----line_5---- node_6 ---- fault_11
 |                    |                     |
source_10          sym_load_4           sym_load_7
```

The 3 nodes are connected in a triangular way by 3 lines.

In [1]:
# some basic imports
import numpy as np

from power_grid_model import (
    CalculationMethod,
    CalculationType,
    ComponentType,
    DatasetType,
    FaultPhase,
    FaultType,
    LoadGenType,
    PowerGridModel,
    ShortCircuitVoltageScaling,
    initialize_array,
)

## Input Dataset

Please refer to [Components](../user_manual/components.md) for detailed explanation of all component types and their input/output attributes.

In [2]:
# node
node = initialize_array(DatasetType.input, ComponentType.node, 3)
node["id"] = np.array([1, 2, 6])
node["u_rated"] = [10.5e3, 10.5e3, 10.5e3]

# line
line = initialize_array(DatasetType.input, ComponentType.line, 3)
line["id"] = [3, 5, 8]
line["from_node"] = [1, 2, 1]
line["to_node"] = [2, 6, 6]
line["from_status"] = [1, 1, 1]
line["to_status"] = [1, 1, 1]
line["r1"] = [0.25, 0.25, 0.25]
line["x1"] = [0.2, 0.2, 0.2]
line["c1"] = [10e-6, 10e-6, 10e-6]
line["tan1"] = [0.0, 0.0, 0.0]
line["i_n"] = [1000, 1000, 1000]

# load
sym_load = initialize_array(DatasetType.input, ComponentType.sym_load, 2)
sym_load["id"] = [4, 7]
sym_load["node"] = [2, 6]
sym_load["status"] = [1, 1]
sym_load["type"] = [LoadGenType.const_power, LoadGenType.const_power]
sym_load["p_specified"] = [20e6, 10e6]
sym_load["q_specified"] = [5e6, 2e6]

# source
source = initialize_array(DatasetType.input, ComponentType.source, 1)
source["id"] = [10]
source["node"] = [1]
source["status"] = [1]
source["u_ref"] = [1.0]

# fault
fault = initialize_array(DatasetType.input, ComponentType.fault, 1)
fault["id"] = [11]
fault["status"] = [1]
fault["fault_object"] = [6]
fault["fault_type"] = [FaultType.three_phase]
fault["fault_phase"] = [FaultPhase.abc]
fault["r_f"] = [0.1]
fault["x_f"] = [0.1]

# all
input_data = {
    ComponentType.node: node,
    ComponentType.line: line,
    ComponentType.sym_load: sym_load,
    ComponentType.source: source,
    ComponentType.fault: fault,
}


**Note: During a single calculation, all types of faults should be similar.**

## Validation (optional)
For efficiency reasons, most of the data is not explicitly validated in the power grid model. However, in most cases, a power flow calculation will fail/crash if the data is invalid. Often with a low level error message that is hard to relate to the original objects. Therfore, it is recommended to always validate your data before constructing a PowerGridModel instance.

The simplest and most effective way to validate your data is by using `assert_valid_input_data()` which will throw an error if it encounters any invalid data. See [Validation Examples](./Validation%20Examples.ipynb) for more detailed information on the validation functions.

In [3]:
from power_grid_model.validation import assert_valid_input_data

assert_valid_input_data(input_data=input_data, calculation_type=CalculationType.short_circuit)

## Construction

The construction of the model is just calling the constructor of `PowerGridModel`.


In [4]:
model = PowerGridModel(input_data)

## One-time Short circuit Calculation

You can call the method `calculate_short_circuit` to do a one-time calculation based on the current network data in the model.

The short circuit calculation has the following settings as arguments:
 * calculation_method: CalculationMethod.iec60909,
 * short_circuit_voltage_scaling: ShortCircuitVoltageScaling.maximum

Currently, there is only one calculation method for short-circuit which calculates as per IEC 60909. The `short_circuit_voltage_scaling` is a scaling of source voltages based on the nominal node voltages.


In [5]:
output_data = model.calculate_short_circuit(
    calculation_method=CalculationMethod.iec60909, short_circuit_voltage_scaling=ShortCircuitVoltageScaling.maximum
)

### Result Dataset

The short circuit calculation results are always asymmetric. This means we cannot convert them to dataframes

The following code prints some result attributs of nodes, faults, and lines in arrays.


In [6]:
print("\n------fault result: id------")
print(output_data[ComponentType.fault]["id"])
print("------fault result: i_f------")
print(output_data[ComponentType.fault]["i_f"])

print("\n------node result: id------")
print(output_data[ComponentType.node]["id"])
print("------node result: u_pu------")
print(output_data[ComponentType.node]["u_pu"])

print("\n------line result: id------")
print(output_data[ComponentType.line]["id"])
print("------line result: u_pu------")
print(output_data[ComponentType.line]["i_from"])


------fault result: id------
[11]
------fault result: i_f------
[[18404.40044631 18404.40044631 18404.40044631]]

------node result: id------
[1 2 6]
------node result: u_pu------
[[1.07539348 1.07539348 1.07539348]
 [0.75227027 0.75227027 0.75227027]
 [0.42934657 0.42934657 0.42934657]]

------line result: id------
[3 5 8]
------line result: u_pu------
[[ 6119.72533835  6119.72533835  6119.72533835]
 [ 6131.26794632  6131.26794632  6131.26794632]
 [12255.79671336 12255.79671336 12255.79671336]]


## Batch Calculations

The batch calculations are mentioned in detail in the [Power Flow Example](./Power%20Flow%20Example.ipynb). Short circuit batch calculations are carried out in similar way.